In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

**Load Data and combine test and train data into 1 DataFrame**

In [3]:
# Load the data, and separate the target
iowa_file_path = "/kaggle/input/home-data-for-ml-course/train.csv"
train_data = pd.read_csv(iowa_file_path)
iowa_file_path = "/kaggle/input/home-data-for-ml-course/test.csv"
test_data = pd.read_csv(iowa_file_path)

df = pd.concat([train_data, test_data])

Data exploration:

In [4]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,1460.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,180921.195890
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129975.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [5]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

**Feature engineering**

It is common knowledge that usually in a city each neighborhood has typically different average price per square feet, which is the first feature we will include:

In [6]:
#First group by neighborhood and calculate mean sale price for each neighborhood
df['NPrice'] = df.groupby(['Neighborhood'])['SalePrice'].transform('mean')

#Calculate mean 1st and 2nd floor square foot for each neighborhood
df['NSF'] = df.groupby(['Neighborhood'])['1stFlrSF'].transform('mean') + df.groupby(['Neighborhood'])['2ndFlrSF'].transform('mean')

#Same for mean LotArea
df['NLA'] = df.groupby(['Neighborhood'])['LotArea'].transform('mean')

# Next devide the mean Price by mean area to get the average price per square foot for each neighborhood
df['NPrice/SF'] = df['NPrice']/df['NSF'] #(df['1stFlrSF']+df['2ndFlrSF'])

#Average Price per Lot Area in each Neighborhood
df['NPrice/LA'] = df['NPrice']/df['NLA']

Next split features into numerical features and categorical features

In [7]:
features = df.select_dtypes('number').columns
#features = features.drop('MSSubClass')

cat_features = df.select_dtypes('object').columns
#cat_features = cat_features.insert(0, 'MSSubClass')

In [8]:
features

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'NPrice', 'NSF', 'NLA',
       'NPrice/SF', 'NPrice/LA'],
      dtype='object')

The features MSSubClass is a categorical feature with categories which are indicated by numbers so I will remove it from the list of numerical features

In [9]:
features = features.drop(['MSSubClass'])
cat_features = cat_features.insert(0, 'MSSubClass')

In [10]:
#These are the categoricl features
cat_features

Index(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')

I don't want to include all categorical features since some may not be very imformative so I pick only those I consider important:

In [11]:
add_features = ['MSZoning', 'RoofMatl', 'CentralAir', 'Utilities','Heating', 'Street','HeatingQC','SaleType','SaleCondition',
               'Electrical', 'Foundation', 'LotShape']

I noticed that 1 datapoint has GarageYrBlt as 2207 which is obviously a mistake and should be 2007, so I fix this in the next

In [12]:
df[df['GarageYrBlt']==2207]
df.loc[df['GarageYrBlt']==2207,'GarageYrBlt'] = 2007

df.GarageYrBlt.describe()

count    2760.000000
mean     1978.040942
std        25.206206
min      1895.000000
25%      1960.000000
50%      1979.000000
75%      2002.000000
max      2010.000000
Name: GarageYrBlt, dtype: float64

Next check how many features of those which I picked have missing values

In [13]:
missing_val_count_by_column = (df[features].isnull().sum())
print('Missing numerical features:')
print(missing_val_count_by_column[missing_val_count_by_column > 0])

missing_val_count_by_column = (df[add_features].isnull().sum())
print('Missing categorical features:')
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Missing numerical features:
LotFrontage      486
MasVnrArea        23
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
GarageYrBlt      159
GarageCars         1
GarageArea         1
SalePrice       1459
dtype: int64
Missing categorical features:
MSZoning      4
Utilities     2
SaleType      1
Electrical    1
dtype: int64


To fill in some missing numerical features I will assume that those that are missing means that the feature does not exist in the house, e.g. if the number of Full Bathrooms in the Basement (BsmtFullBath) is missing I assume that it whould be 0 (i.e. it does not exist). Similiarly if the GarageArea is missing I assume that there is no Garage.

Furthermore it makes sense to combine similiar features like BsmtFullBath, BsmtHalfBath, FullBath and HalfBath into 1 single feature called TotalBaths. Similiarly I combine 1stFlrSF with 2ndFlrSF and TotalBsmtSF into 1 feature called SF. I do the same for PorchSF.

Finally I create 1 more feature called PriceEst which will be a naive price estimate based on the mean price per square foot times square foot house area and mean price per lot area times the lot area.

In [14]:
df['BsmtFullBath'] = df['BsmtFullBath'].fillna(0)
df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(0)

df['LotFrontage'] = df['LotFrontage'].fillna(0)
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
df['GarageArea'] = df['GarageArea'].fillna(0)

#combine all baths together
df['TotalBath'] = df['BsmtFullBath'] + 0.5*df['BsmtHalfBath'] + df['FullBath'] + 0.5*df['HalfBath']
#combine all sq ft
df['SF'] = (df['1stFlrSF']+df['2ndFlrSF'] + df['TotalBsmtSF'])
#combine porch area
df['PorchSF'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
##
df['PriceEst'] = 0.5*(df['NPrice/SF']*df['SF'] + df['NPrice/LA']*df['LotArea']) + df['MiscVal']


#Drop all this features because they won't be used

features = features.drop(['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'BsmtHalfBath', 'GarageCars', 'TotalBsmtSF',
'2ndFlrSF', '1stFlrSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
                         'HalfBath', 'MiscVal', 'NPrice/LA'])

In [15]:
#Include new features in the feature list

features = features.insert(0, 'TotalBath')
features = features.insert(0, 'PorchSF')
features = features.insert(0, 'PriceEst')

print(features)

Index(['PriceEst', 'PorchSF', 'TotalBath', 'Id', 'LotFrontage', 'LotArea',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'LowQualFinSF', 'GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF',
       'PoolArea', 'MoSold', 'YrSold', 'SalePrice', 'NPrice', 'NSF', 'NLA',
       'NPrice/SF'],
      dtype='object')


Next I will transform the features including year like YearBuilt to show how old the house is instead, i.e. if Year built is 2006 then the age is 2010-2006=4. I do this same transformation with YrSold, GarageYrBlt, YearRemodAdd

In [16]:
#features with year:
#transform to indicate how many years passed since, YearBuilt, YearRemodAdded, GarageYrBlt, YrSold
df['YearBuilt'] = 2010 - df['YearBuilt'] #--> age of building
df['YearRemodAdd'] = 2010 - df['YearRemodAdd']
df['GarageYrBlt'] = 2010 - df['GarageYrBlt']
df['YrSold'] = 2010 - df['YrSold']

Include 1 more feature which give the average price per MSSubClass, which indicates the type of building which has big impact on its price

In [17]:
df['MSSubClassPrice'] = df.groupby(['MSSubClass'])['SalePrice'].transform('mean')

features = features.insert(0, 'MSSubClassPrice')

Add all additional features into the list of features

In [18]:
for i in add_features:
    features = features.insert(0, i)

In [19]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,NPrice,NSF,NLA,NPrice/SF,NPrice/LA,TotalBath,SF,PorchSF,PriceEst,MSSubClassPrice
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2918.000000,2919.000000,2.918000e+03,2918.000000
mean,1460.000000,57.137718,57.766701,10168.114080,6.089072,5.564577,38.687222,25.735526,102.201312,441.423235,...,179951.593799,1496.065433,10168.114080,117.960606,19.221707,2.218397,2548.048663,89.249743,2.442047e+05,180450.485934
std,842.787043,42.517628,33.481636,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,59002.154605,269.602824,3104.864682,19.816110,9.039221,0.808840,804.677866,107.902805,1.120620e+05,39014.840915
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,98576.470588,1066.702703,1840.633333,84.917986,8.556609,1.000000,334.000000,0.000000,6.178848e+04,95829.724638
25%,730.500000,20.000000,43.000000,7478.000000,5.000000,5.000000,9.000000,6.000000,0.000000,0.000000,...,136793.135135,1288.516930,8826.473118,101.480404,14.527033,1.500000,2000.500000,0.000000,1.699894e+05,143302.972222
50%,1460.000000,50.000000,63.000000,9453.000000,6.000000,5.000000,37.000000,17.000000,0.000000,368.500000,...,156270.122449,1415.497908,10039.702032,115.797707,16.211200,2.000000,2450.000000,50.000000,2.205196e+05,185224.811567
75%,2189.500000,70.000000,78.000000,11570.000000,7.000000,6.000000,56.500000,45.000000,164.000000,733.000000,...,197965.773333,1620.575758,11342.369697,132.319462,19.797067,2.500000,2991.750000,136.000000,2.887254e+05,200779.080460
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,138.000000,60.000000,1600.000000,5644.000000,...,335295.317073,2480.633803,24842.250000,162.804667,58.597912,7.000000,11752.000000,1207.000000,1.724567e+06,239948.501672


**OneHotEncode categorical features**

In [20]:
add_features

['MSZoning',
 'RoofMatl',
 'CentralAir',
 'Utilities',
 'Heating',
 'Street',
 'HeatingQC',
 'SaleType',
 'SaleCondition',
 'Electrical',
 'Foundation',
 'LotShape']

In [21]:
features

Index(['LotShape', 'Foundation', 'Electrical', 'SaleCondition', 'SaleType',
       'HeatingQC', 'Street', 'Heating', 'Utilities', 'CentralAir', 'RoofMatl',
       'MSZoning', 'MSSubClassPrice', 'PriceEst', 'PorchSF', 'TotalBath', 'Id',
       'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'LowQualFinSF', 'GrLivArea', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageArea', 'WoodDeckSF', 'PoolArea', 'MoSold', 'YrSold', 'SalePrice',
       'NPrice', 'NSF', 'NLA', 'NPrice/SF'],
      dtype='object')

In [22]:
df = pd.get_dummies(df[features], columns=add_features, drop_first=True)

Check which featues still have missing values

In [23]:
missing_val_count_by_column = (df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSSubClassPrice       1
PriceEst              1
SalePrice          1459
dtype: int64


In [24]:
df['PriceEst'] = df['PriceEst'].fillna(df['PriceEst'].median())
df['MSSubClassPrice'] = df['MSSubClassPrice'].fillna(df['PriceEst'].median())

**Split dataFrame df back to train and test set (competion set)**

In [25]:
dtrain = df[df['SalePrice'].notna()]
dtest = df[df['SalePrice'].isna()]

In [26]:
##
y_train = dtrain['SalePrice']
train = dtrain.drop('SalePrice', axis=1)

test = dtest.drop('SalePrice', axis=1)

#Use StandardScalar to standartise features:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

To choose a good model and estimate its mean squared error split train data into training set and validation set:

In [27]:
X_t, X_v, y_t, y_v = train_test_split(train, y_train, test_size=0.1, random_state=10)

scorer1 = make_scorer(mean_squared_error)

**Build model**

In [28]:
#from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [29]:
#rscv = RandomizedSearchCV(
 #           estimator=XGBRegressor(),
  #          param_distributions={"learning_rate": (0.01, 0.05, 0.1),
   #                     "max_depth": [4, 8, 12],
    #                    "min_child_weight": [1, 3, 5],
     #                   "gamma":[ 0.0, 0.1, 0.2],
      #                 "n_estimators":[100]},
       #     cv=3, n_iter=20, random_state=10,scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

#rscv.fit(X_t, y_t)
#print(rscv.best_params_)
#print(-1*np.sqrt(rscv.best_score_))

In [30]:
#gscv = GridSearchCV(
 #           estimator=XGBRegressor(),
  #          param_grid={"learning_rate": [0.1],
   #                     "max_depth": [4],
    #                    "min_child_weight": [5],
     #                   "gamma":[0.1],
      #                  #"colsample_bytree":[ 0.3, 0.4],
       #                "n_estimators":[1000, 2000]},
        #    cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

#grid_result = gscv.fit(X_t, y_t)

In [31]:
#print(-1*grid_result.best_score_)
#print(grid_result.best_params_)

In [32]:
mod = XGBRegressor(n_estimators=1000, max_depth=4, objective='reg:squarederror', gamma=10, learning_rate=0.1)

#get cross validation score of model
score = cross_val_score(mod, X_t, y_t, cv=5, scoring=scorer1)
print(np.mean(np.sqrt(score)))

#get score on validation set
mod.fit(X_t, y_t)
print(np.sqrt(scorer1(mod, X_v, y_v)))

29173.108821717637
22063.168905650487


**Train final model on full train data**

In [33]:
X = np.concatenate((X_t, X_v), axis=0)
y = np.concatenate((y_t, y_v), axis=0)

In [34]:
X.shape

(1460, 75)

In [35]:
xgb = XGBRegressor(n_estimators=1000, max_depth=4, objective='reg:squarederror', gamma=10, learning_rate=0.1)
xgb.fit(X, y)

print(np.sqrt(scorer1(xgb, X, y)))

1592.4579466291914


**Predict test data for competition and save results**

In [36]:
predictions = xgb.predict(test)

In [37]:
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predictions})
output.to_csv('submission.csv', index=False)

In [38]:
output

,Id,SalePrice
0,1461,136163.703125
1,1462,166444.953125
2,1463,170419.890625
3,1464,192444.765625
4,1465,176026.187500
...,...,...
1454,2915,88623.507812
1455,2916,87193.328125
1456,2917,131975.296875
1457,2918,120071.640625
